# MIMIC IV Data Medical Profiles
Generate patient clinical profile used to develop search query for relevant clinical trials.
- Estimate age from de-identified data
- Retrieve key relevant conditions and terms
- Aggregate information from recent radiology reports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Load Discharge Data
Patient clinical admission-to-discharge notes

In [ ]:
dir = ''

In [ ]:
# Load data
df_discharge = pd.read_csv(dir + '/discharge.csv')

# Convert charttime to pandas datetime
df_discharge['charttime'] = pd.to_datetime(df_discharge['charttime'], format='%Y-%m-%d %H:%M:%S')

# Drop fields
df_discharge = df_discharge.drop(['note_type', 'note_seq', 'storetime'], axis=1)

display(df_discharge.head())
display(df_discharge.info())

,note_id,subject_id,hadm_id,charttime,text
0,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,2180-06-27,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,2180-07-25,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,2180-08-07,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,2160-11-25,\nName: ___ Unit No: __...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331793 entries, 0 to 331792
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   note_id     331793 non-null  object        
 1   subject_id  331793 non-null  int64         
 2   hadm_id     331793 non-null  int64         
 3   charttime   331793 non-null  datetime64[ns]
 4   text        331793 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 12.7+ MB


None

# 2. ICD

In [ ]:
# Load data
df_icd = pd.read_csv(dir + '/diagnoses_icd.csv')

# Rename for clarity
df_icd = df_icd.rename(columns={'seq_num': 'icd_priority'})

df_icd.head()

,subject_id,hadm_id,icd_priority,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [ ]:
# Retain only priority diagnosis codes

n_priority = 5  #TODO

# Filter to retain priority codes
df_icd = df_icd[df_icd.icd_priority <= 5]

In [ ]:
# Join
df_discharge_v1 = pd.merge(df_discharge,
                            df_icd,
                            on=['subject_id', 'hadm_id'], how='left')

df_discharge_v1

,note_id,subject_id,hadm_id,charttime,text,icd_priority,icd_code,icd_version
0,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,1.0,5723,9.0
1,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,2.0,78959,9.0
2,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,3.0,5715,9.0
3,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,4.0,07070,9.0
4,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,5.0,496,9.0
...,...,...,...,...,...,...,...,...
1591605,19999987-DS-2,19999987,23865745,2145-11-11,\nName: ___ Unit No: __...,1.0,431,9.0
1591606,19999987-DS-2,19999987,23865745,2145-11-11,\nName: ___ Unit No: __...,2.0,3485,9.0
1591607,19999987-DS-2,19999987,23865745,2145-11-11,\nName: ___ Unit No: __...,3.0,20280,9.0
1591608,19999987-DS-2,19999987,23865745,2145-11-11,\nName: ___ Unit No: __...,4.0,5849,9.0


## Load ICD Code Descriptions

In [ ]:
def read_icd_descriptions(file_path):
  '''
  Reads ICD desc files txt and returns lookup table. Each entry is a code
  and description, with everything separated by non-delimiting white space.
  Encoding is latin-1.
  '''

  codes = []
  descriptions = []


  with open(file_path, 'r', encoding='latin-1') as f:
      for line in f:
          parts = line.strip().split(maxsplit=1)  # Strip whitespace and split
          if parts:
              codes.append(parts[0])
              descriptions.append(parts[1] if len(parts) > 1 else "")
          else:
              codes.append("")
              descriptions.append("")

  return pd.DataFrame({'icd_code': codes, 'icd_description': descriptions})

In [ ]:
# Read ICD 9
ICD9_desc_2015 = read_icd_descriptions(dir + '/CMS32_DESC_LONG_DX_2015.txt')
# Read ICD 10
ICD10_desc_2023 = read_icd_descriptions(dir + '/icd10cm_codes_2023.txt')

display(ICD9_desc_2015)
display(ICD10_desc_2023)


,icd_code,icd_description
0,0010,Cholera due to vibrio cholerae
1,0011,Cholera due to vibrio cholerae el tor
2,0019,"Cholera, unspecified"
3,0020,Typhoid fever
4,0021,Paratyphoid fever A
...,...,...
14562,V9129,"Quadruplet gestation, unable to determine numb..."
14563,V9190,"Other specified multiple gestation, unspecifie..."
14564,V9191,"Other specified multiple gestation, with two o..."
14565,V9192,"Other specified multiple gestation, with two o..."


,icd_code,icd_description
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A009,"Cholera, unspecified"
3,A0100,"Typhoid fever, unspecified"
4,A0101,Typhoid meningitis
...,...,...
73669,Z9981,Dependence on supplemental oxygen
73670,Z9989,Dependence on other enabling machines and devices
73671,U070,Vaping-related disorder
73672,U071,COVID-19


## Translate ICD
Add descriptions using ICD lookup
- Using latest version, 2015, for ICD 9
- Using active version as of the end date of dataset, 2023, for ICD 10

In [ ]:
# Append descriptions with respect to ICD version

df_discharge_v1_icd9 = pd.merge(df_discharge_v1[df_discharge_v1['icd_version'] == 9],
         ICD9_desc_2015,
         on='icd_code',
         how='left')

df_discharge_v1_icd10 = pd.merge(df_discharge_v1[df_discharge_v1['icd_version'] == 10],
         ICD10_desc_2023,
         on='icd_code',
         how='left')

# Cases with no ICD codes
df_discharge_v1_icdnone = df_discharge_v1[(df_discharge_v1['icd_version'] != 9) & (df_discharge_v1['icd_version'] != 10)]

# Discharge notes with joined icd decscriptions
df_discharge_v2 = pd.concat([df_discharge_v1_icd9, df_discharge_v1_icd10, df_discharge_v1_icdnone], ignore_index=True)

# Drop
df_discharge_v2 = df_discharge_v2.drop(['icd_code', 'icd_version'], axis=1)

# Delete df
del df_discharge_v1
del df_discharge_v1_icd9
del df_discharge_v1_icd10
del df_discharge_v1_icdnone

df_discharge_v2

,note_id,subject_id,hadm_id,charttime,text,icd_priority,icd_description
0,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,1.0,Portal hypertension
1,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,2.0,Other ascites
2,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,3.0,Cirrhosis of liver without mention of alcohol
3,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,4.0,Unspecified viral hepatitis C without hepatic ...
4,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,5.0,"Chronic airway obstruction, not elsewhere clas..."
...,...,...,...,...,...,...,...
1591605,19965819-DS-5,19965819,28425094,2164-02-10,\nName: ___ Unit No: ___...,NaN,NaN
1591606,19965819-DS-6,19965819,20088669,2164-02-16,\nName: ___ Unit No: ___...,NaN,NaN
1591607,19973723-DS-6,19973723,27287088,2150-12-07,\nName: ___ Unit No: ___\n...,NaN,NaN
1591608,19979521-DS-11,19979521,24906806,2200-09-08,\nName: ___ Unit No: __...,NaN,NaN


## Aggregate ICD

In [ ]:
# Sort by subject_hadm and priority
df_aggregate_icd_desc = df_discharge_v2.sort_values(by=['subject_id', 'hadm_id', 'icd_priority'])

# Aggregate diagnosis texts into a list, ordered by priority
df_aggregate_icd_desc = df_aggregate_icd_desc.groupby(['subject_id', 'hadm_id'])['icd_description'].apply(list).reset_index(name='ordered_diagnoses')

df_aggregate_icd_desc

,subject_id,hadm_id,ordered_diagnoses
0,10000032,22595853,"[Portal hypertension, Other ascites, Cirrhosis..."
1,10000032,22841357,[Unspecified viral hepatitis C with hepatic co...
2,10000032,25742920,[Chronic hepatitis C without mention of hepati...
3,10000032,29079034,"[Other iatrogenic hypotension, Chronic hepatit..."
4,10000084,23052089,"[Neurocognitive disorder with Lewy bodies, Dem..."
...,...,...,...
331788,19999828,25744818,"[Infection following a procedure, superficial ..."
331789,19999828,29734428,[Disruption of external operation (surgical) w...
331790,19999840,21033226,"[Grand mal status, Acute respiratory failure, ..."
331791,19999840,26071774,"[Cerebral artery occlusion, unspecified with c..."


In [ ]:
# Join aggregate table back to main discharge table
df_discharge_v3 = pd.merge(df_discharge,
                            df_aggregate_icd_desc,
                            on=['subject_id', 'hadm_id'], how='left')

# delete df
del df_discharge_v2
del df_aggregate_icd_desc


df_discharge_v3

,note_id,subject_id,hadm_id,charttime,text,ordered_diagnoses
0,10000032-DS-21,10000032,22595853,2180-05-07,\nName: ___ Unit No: _...,"[Portal hypertension, Other ascites, Cirrhosis..."
1,10000032-DS-22,10000032,22841357,2180-06-27,\nName: ___ Unit No: _...,[Unspecified viral hepatitis C with hepatic co...
2,10000032-DS-23,10000032,29079034,2180-07-25,\nName: ___ Unit No: _...,"[Other iatrogenic hypotension, Chronic hepatit..."
3,10000032-DS-24,10000032,25742920,2180-08-07,\nName: ___ Unit No: _...,[Chronic hepatitis C without mention of hepati...
4,10000084-DS-17,10000084,23052089,2160-11-25,\nName: ___ Unit No: __...,"[Neurocognitive disorder with Lewy bodies, Dem..."
...,...,...,...,...,...,...
331788,19999828-DS-6,19999828,29734428,2147-08-04,\nName: ___ Unit No: ___...,[Disruption of external operation (surgical) w...
331789,19999828-DS-7,19999828,25744818,2149-01-18,\nName: ___ Unit No: ___...,"[Infection following a procedure, superficial ..."
331790,19999840-DS-20,19999840,26071774,2164-07-28,\nName: ___ Unit No: ___\...,"[Cerebral artery occlusion, unspecified with c..."
331791,19999840-DS-21,19999840,21033226,2164-09-17,\nName: ___ Unit No: ___\...,"[Grand mal status, Acute respiratory failure, ..."


# 3. Patient Demographics

In [ ]:
# Load data
df_patients = pd.read_csv(dir + '/patients.csv')

df_patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000058,F,33,2168,2020 - 2022,NaN
3,10000068,F,19,2160,2008 - 2010,NaN
4,10000084,M,72,2160,2017 - 2019,2161-02-13
...,...,...,...,...,...,...
364622,19999828,F,46,2147,2017 - 2019,NaN
364623,19999829,F,28,2186,2008 - 2010,NaN
364624,19999840,M,58,2164,2008 - 2010,2164-09-17
364625,19999914,F,49,2158,2017 - 2019,NaN


In [ ]:
# Assume first year in anchor group is anchor origin year
df_patients['anchor_origin_year'] = df_patients['anchor_year_group'].str[:4].astype(int)
# Estimate birth year
df_patients['birth_year'] = df_patients['anchor_origin_year'] - df_patients['anchor_age']

# Calculate the offset per patient
df_patients['offset'] = df_patients['anchor_year'] - df_patients['anchor_origin_year']

# Display
display(df_patients)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,anchor_origin_year,birth_year,offset
0,10000032,F,52,2180,2014 - 2016,2180-09-09,2014,1962,166
1,10000048,F,23,2126,2008 - 2010,NaN,2008,1985,118
2,10000058,F,33,2168,2020 - 2022,NaN,2020,1987,148
3,10000068,F,19,2160,2008 - 2010,NaN,2008,1989,152
4,10000084,M,72,2160,2017 - 2019,2161-02-13,2017,1945,143
...,...,...,...,...,...,...,...,...,...
364622,19999828,F,46,2147,2017 - 2019,NaN,2017,1971,130
364623,19999829,F,28,2186,2008 - 2010,NaN,2008,1980,178
364624,19999840,M,58,2164,2008 - 2010,2164-09-17,2008,1950,156
364625,19999914,F,49,2158,2017 - 2019,NaN,2017,1968,141


In [ ]:
# Merge to discharge, creating table of discharges with patient demographics
df_discharge_v4 = pd.merge(df_discharge_v3,
                           df_patients.loc[:, ['subject_id', 'gender', 'birth_year', 'offset']],
                           on='subject_id', how='left')

# Drop fields missing basic demographics
df_discharge_v4 = df_discharge_v4.dropna(subset=['gender', 'birth_year', 'offset'])
# Handle undesired type behavior
df_discharge_v4['birth_year'] = df_discharge_v4['birth_year'].astype(int)
df_discharge_v4['offset'] = df_discharge_v4['offset'].astype(int)
display(df_discharge_v4.info())

# Create new field indicating estimated true charttime
df_discharge_v4['date'] = df_discharge_v4.apply(lambda row: row['charttime'] - pd.DateOffset(years=row['offset']), axis=1)

# Calculate patient estimated age at time of encounter
df_discharge_v4['age'] = df_discharge_v4['date'].dt.year - df_discharge_v4['birth_year']

# Drop columns
df_discharge_v4 = df_discharge_v4.drop(['birth_year', 'offset', 'charttime'], axis=1)

# delete df
del df_patients
del df_discharge_v3

# Display
df_discharge_v4

<class 'pandas.core.frame.DataFrame'>
Index: 331761 entries, 0 to 331792
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   note_id            331761 non-null  object        
 1   subject_id         331761 non-null  int64         
 2   hadm_id            331761 non-null  int64         
 3   charttime          331761 non-null  datetime64[ns]
 4   text               331761 non-null  object        
 5   ordered_diagnoses  331761 non-null  object        
 6   gender             331761 non-null  object        
 7   birth_year         331761 non-null  int64         
 8   offset             331761 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 25.3+ MB


None

,note_id,subject_id,hadm_id,text,ordered_diagnoses,gender,date,age
0,10000032-DS-21,10000032,22595853,\nName: ___ Unit No: _...,"[Portal hypertension, Other ascites, Cirrhosis...",F,2014-05-07,52
1,10000032-DS-22,10000032,22841357,\nName: ___ Unit No: _...,[Unspecified viral hepatitis C with hepatic co...,F,2014-06-27,52
2,10000032-DS-23,10000032,29079034,\nName: ___ Unit No: _...,"[Other iatrogenic hypotension, Chronic hepatit...",F,2014-07-25,52
3,10000032-DS-24,10000032,25742920,\nName: ___ Unit No: _...,[Chronic hepatitis C without mention of hepati...,F,2014-08-07,52
4,10000084-DS-17,10000084,23052089,\nName: ___ Unit No: __...,"[Neurocognitive disorder with Lewy bodies, Dem...",M,2017-11-25,72
...,...,...,...,...,...,...,...,...
331788,19999828-DS-6,19999828,29734428,\nName: ___ Unit No: ___...,[Disruption of external operation (surgical) w...,F,2017-08-04,46
331789,19999828-DS-7,19999828,25744818,\nName: ___ Unit No: ___...,"[Infection following a procedure, superficial ...",F,2019-01-18,48
331790,19999840-DS-20,19999840,26071774,\nName: ___ Unit No: ___\...,"[Cerebral artery occlusion, unspecified with c...",M,2008-07-28,58
331791,19999840-DS-21,19999840,21033226,\nName: ___ Unit No: ___\...,"[Grand mal status, Acute respiratory failure, ...",M,2008-09-17,58


# 4. Load Radiology Data
Radiology notes for patients. Not always associated with hospital admissions.

In [ ]:
# Load data
df_rad = pd.read_csv(dir + '/radiology.csv')

# Convert charttime to pandas datetime
df_rad['charttime'] = pd.to_datetime(df_rad['charttime'], format='%Y-%m-%d %H:%M:%S')

# Convert NaN hadm_id to -1, then cast to int
df_rad['hadm_id'] = pd.to_numeric(df_rad['hadm_id'], errors='coerce').fillna(-1).astype(int)

# Rename columns to indicate radiology-specific
df_rad = df_rad.rename(columns={'charttime': 'rad_charttime', 'text': 'rad_desc'})

display(df_rad)
display(df_rad.info())

,note_id,subject_id,hadm_id,note_type,note_seq,rad_charttime,storetime,rad_desc
0,10000032-RR-14,10000032,22595853,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
1,10000032-RR-15,10000032,22595853,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
2,10000032-RR-16,10000032,22595853,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi..."
3,10000032-RR-18,10000032,-1,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,EXAMINATION: Ultrasound-guided paracentesis.\...
4,10000032-RR-20,10000032,-1,RR,20,2180-07-08 13:18:00,2180-07-08 14:15:00,EXAMINATION: Paracentesis\n\nINDICATION: ___...
...,...,...,...,...,...,...,...,...
2321350,19999987-RR-17,19999987,23865745,RR,17,2145-11-02 22:37:00,2145-11-03 18:55:00,"HISTORY: ___, with left occipital bleeding. ..."
2321351,19999987-RR-18,19999987,23865745,RR,18,2145-11-03 04:35:00,2145-11-03 10:46:00,INDICATION: ___ female intubated for head ble...
2321352,19999987-RR-19,19999987,23865745,RR,19,2145-11-03 16:40:00,2145-11-04 08:36:00,HISTORY: ___ woman with left occipital hemorr...
2321353,19999987-RR-20,19999987,23865745,RR,20,2145-11-04 05:10:00,2145-11-04 08:58:00,PORTABLE CHEST OF ___\n\nCOMPARISON: ___ radi...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2321355 entries, 0 to 2321354
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   note_id        object        
 1   subject_id     int64         
 2   hadm_id        int64         
 3   note_type      object        
 4   note_seq       int64         
 5   rad_charttime  datetime64[ns]
 6   storetime      object        
 7   rad_desc       object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 141.7+ MB


None

## Aggregate Rad Reports

In [ ]:
# Join radiology reports by subject and admission
df_aggregate_rad_desc = pd.merge(df_discharge_v4,
                            df_rad.loc[:, ['subject_id', 'hadm_id', 'rad_charttime', 'rad_desc']],
                            on=['subject_id', 'hadm_id'], how='left')

# Sort by subject_hadm and priority
df_aggregate_rad_desc = df_aggregate_rad_desc.sort_values(['subject_id', 'hadm_id', 'rad_charttime'])

# Retain only the top n
n_rad_priority = 5  #TODO
df_aggregate_rad_desc = df_aggregate_rad_desc.groupby(['subject_id', 'hadm_id']).head(n_rad_priority).reset_index(drop=True)

# Aggregate diagnosis texts into a list, ordered by priority
df_aggregate_rad_desc = df_aggregate_rad_desc.groupby(['subject_id', 'hadm_id'])['rad_desc'].apply(list).reset_index(name='ordered_rad')

df_aggregate_rad_desc.head()

,subject_id,hadm_id,ordered_rad
0,10000032,22595853,[EXAMINATION: CHEST (PA AND LAT)\n\nINDICATIO...
1,10000032,22841357,[EXAMINATION: LIVER OR GALLBLADDER US (SINGLE...
2,10000032,25742920,[INDICATION: ___ year old woman with cirrhosi...
3,10000032,29079034,[EXAMINATION: CT HEAD W/O CONTRAST\n\nINDICAT...
4,10000084,23052089,[nan]


In [ ]:
# Join aggregate rad reports to discharge table
df_discharge_v5 = pd.merge(df_discharge_v4,
                            df_aggregate_rad_desc,
                            on=['subject_id', 'hadm_id'], how='left')

df_discharge_v5.head()

,note_id,subject_id,hadm_id,text,ordered_diagnoses,gender,date,age,ordered_rad
0,10000032-DS-21,10000032,22595853,\nName: ___ Unit No: _...,"[Portal hypertension, Other ascites, Cirrhosis...",F,2014-05-07,52,[EXAMINATION: CHEST (PA AND LAT)\n\nINDICATIO...
1,10000032-DS-22,10000032,22841357,\nName: ___ Unit No: _...,[Unspecified viral hepatitis C with hepatic co...,F,2014-06-27,52,[EXAMINATION: LIVER OR GALLBLADDER US (SINGLE...
2,10000032-DS-23,10000032,29079034,\nName: ___ Unit No: _...,"[Other iatrogenic hypotension, Chronic hepatit...",F,2014-07-25,52,[EXAMINATION: CT HEAD W/O CONTRAST\n\nINDICAT...
3,10000032-DS-24,10000032,25742920,\nName: ___ Unit No: _...,[Chronic hepatitis C without mention of hepati...,F,2014-08-07,52,[INDICATION: ___ year old woman with cirrhosi...
4,10000084-DS-17,10000084,23052089,\nName: ___ Unit No: __...,"[Neurocognitive disorder with Lewy bodies, Dem...",M,2017-11-25,72,[nan]


# 5. Clean

In [ ]:
# Reorder columns for clarity
display(df_discharge_v5.columns)

df_discharge_v5 = df_discharge_v5.reindex(columns=[
    'note_id',
    'subject_id',
    'hadm_id',
    'gender',
    'age',
    'date',
    'text',
    'ordered_rad',
    'ordered_diagnoses'])

df_discharge_v5

Index(['note_id', 'subject_id', 'hadm_id', 'text', 'ordered_diagnoses',
       'gender', 'date', 'age', 'ordered_rad'],
      dtype='object')

,note_id,subject_id,hadm_id,gender,age,date,text,ordered_rad,ordered_diagnoses
0,10000032-DS-21,10000032,22595853,F,52,2014-05-07,\nName: ___ Unit No: _...,[EXAMINATION: CHEST (PA AND LAT)\n\nINDICATIO...,"[Portal hypertension, Other ascites, Cirrhosis..."
1,10000032-DS-22,10000032,22841357,F,52,2014-06-27,\nName: ___ Unit No: _...,[EXAMINATION: LIVER OR GALLBLADDER US (SINGLE...,[Unspecified viral hepatitis C with hepatic co...
2,10000032-DS-23,10000032,29079034,F,52,2014-07-25,\nName: ___ Unit No: _...,[EXAMINATION: CT HEAD W/O CONTRAST\n\nINDICAT...,"[Other iatrogenic hypotension, Chronic hepatit..."
3,10000032-DS-24,10000032,25742920,F,52,2014-08-07,\nName: ___ Unit No: _...,[INDICATION: ___ year old woman with cirrhosi...,[Chronic hepatitis C without mention of hepati...
4,10000084-DS-17,10000084,23052089,M,72,2017-11-25,\nName: ___ Unit No: __...,[nan],"[Neurocognitive disorder with Lewy bodies, Dem..."
...,...,...,...,...,...,...,...,...,...
331756,19999828-DS-6,19999828,29734428,F,46,2017-08-04,\nName: ___ Unit No: ___...,[INDICATION: ___ year old woman with new righ...,[Disruption of external operation (surgical) w...
331757,19999828-DS-7,19999828,25744818,F,48,2019-01-18,\nName: ___ Unit No: ___...,[EXAMINATION: UNILAT LOWER EXT VEINS RIGHT\n\...,"[Infection following a procedure, superficial ..."
331758,19999840-DS-20,19999840,26071774,M,58,2008-07-28,\nName: ___ Unit No: ___\...,[HISTORY: Seizure and fever.\n\nFINDINGS: Fr...,"[Cerebral artery occlusion, unspecified with c..."
331759,19999840-DS-21,19999840,21033226,M,58,2008-09-17,\nName: ___ Unit No: ___\...,[INDICATION: ___ man with seizure and elevate...,"[Grand mal status, Acute respiratory failure, ..."


# Sample for annotation
- Sample 50 rows for annotation

In [ ]:
annotate_discharge_sample = df_discharge_v5.sample(50, random_state=35).reset_index(drop=True)
display(annotate_discharge_sample)

# Save as csv
annotate_discharge_sample.to_csv(dir + '/annotate_discharge_sample.csv', index=True)

,note_id,subject_id,hadm_id,gender,age,date,text,ordered_rad,ordered_diagnoses
0,18367270-DS-6,18367270,28706071,F,30,2017-10-01,\nName: ___ Unit No: ___\n \...,[EXAMINATION: MR HEAD W AND W/O CONTRAST T___...,[Other specified congenital malformations of b...
1,18234943-DS-7,18234943,22128361,F,53,2014-05-22,\nName: ___ Unit No: __...,[HISTORY: Cough and fever \n\nCOMPARISON: No...,"[Asthma, unspecified type, unspecified, Hypoxe..."
2,19011334-DS-6,19011334,28892047,M,78,2014-07-30,\nName: ___ Unit No: ...,[nan],[Calculus of bile duct without cholangitis or ...
3,18826698-DS-15,18826698,20922553,M,60,2012-02-01,\nName: ___ Unit No: ___...,[nan],"[Abdominal pain, periumbilic, Localization-rel..."
4,14066157-DS-39,14066157,26888351,M,68,2009-07-20,\nName: ___ Unit No: ___\n ...,[INDICATION: Rigors and fever.\n\nCOMPARISON:...,"[Acute kidney failure, unspecified, Acute on c..."
5,13973494-DS-17,13973494,26551541,M,63,2018-04-06,\nName: ___ Unit No: ___\n \n...,"[INDICATION: ___ with unsteadiness, vomiting...","[Other peripheral vertigo, unspecified ear, Un..."
6,18511092-DS-9,18511092,28348890,M,61,2017-09-25,\nName: ___ Unit No: ___...,[EXAMINATION: CT HEAD W/O CONTRAST\n\nINDICAT...,[ST elevation (STEMI) myocardial infarction in...
7,18922533-DS-16,18922533,23499660,F,81,2017-02-19,\nName: ___ Unit No: __...,[nan],"[Other nontraumatic intracerebral hemorrhage, ..."
8,12018806-DS-3,12018806,22957216,M,35,2014-09-18,\nName: ___ Unit No: ___\...,[EXAMINATION: LIVER OR GALLBLADDER US (SINGLE...,"[Alcoholic cirrhosis of liver with ascites, Se..."
9,17414848-DS-15,17414848,20465956,F,78,2017-11-08,\nName: ___ Unit No: ___...,[EXAMINATION: CHEST (AP AND LAT)\n\nINDICATIO...,"[Sepsis due to Escherichia coli [E. coli], End..."
